In [1]:
# import random as rnd
# import secrets as scr

from datetime import datetime
from random import SystemRandom
from itertools import repeat

rnd = SystemRandom()


class Lotteria:
    def __init__(self,
                 max_num=90,
                 max_sup=90,
                 length=6,
                 superstar=1):
        self.max_numbers = max_num
        self.max_superstar = max_sup
        self.length = length
        self.superstar = superstar

    @staticmethod
    def pallottoliere(num):
        while True:
            yield rnd.randint(1, num)

    @staticmethod
    def choice(length, num):
        numbers = list(range(1, num+1))

        combo = []
        for _ in range(length):
            rnd.shuffle(numbers)
            single = rnd.choice(numbers)
            combo.append(single)
            numbers.remove(single)

        return sorted(combo)

    @staticmethod
    def sample(length, num):
        numbers = list(range(1, num+1))

        return sorted(rnd.sample(numbers, k=length))

    def int(self, length, num):
        combo = set()
        while len(combo) < length:
            combo.add(next(self.pallottoliere(num)))

        return sorted(combo)

    @staticmethod
    def many(picker, length, num):
        #samples = [picker(length, num) for _ in range(num)]
        samples = list(repeat(picker(length, num), num))
        rnd.shuffle(samples)
       
        return sorted(rnd.choice(samples))

    def __call__(self, backend='sample', many=True):
        
        if backend == 'choice':
            self.backend = 'choice'
            picker = self.choice

        if backend == 'sample':
            self.backend = 'sample'
            picker = self.sample

        if backend == 'int':
            self.backend = 'int'
            picker = self.int

        if not many:
            combo = picker(self.length, self.max_numbers) # type: ignore

            if self.superstar:
                super = picker(self.superstar, self.max_superstar) # type: ignore

        else:
            combo = self.many(picker, self.length, self.max_numbers) # type: ignore

            if self.superstar:
                super = self.many(picker, self.superstar, self.max_superstar) # type: ignore

        now = datetime.now().strftime("%d/%m/%Y %H:%M")

        print(f'Estrazione del: {now}', '\nnumbers combo:', *combo)
        if self.superstar:
            print('Superstar:', *sorted(super)) # type: ignore


In [2]:
superenalotto = Lotteria()


In [3]:
for b in ['choice', 'int', 'sample']:
    print('###', b, '###')
    superenalotto(backend=b, many=False)

print('\n################# many #########################\n')

for b in ['choice', 'int', 'sample']:
    print('###', b, '###')
    superenalotto(backend=b, many=True)

### choice ###
Estrazione del: 30/05/2023 12:20 
numbers combo: 51 55 68 80 81 84
Superstar: 51
### int ###
Estrazione del: 30/05/2023 12:20 
numbers combo: 23 66 69 70 81 83
Superstar: 32
### sample ###
Estrazione del: 30/05/2023 12:20 
numbers combo: 9 22 33 38 75 88
Superstar: 86

################# many #########################

### choice ###
Estrazione del: 30/05/2023 12:20 
numbers combo: 17 21 32 51 56 71
Superstar: 49
### int ###
Estrazione del: 30/05/2023 12:20 
numbers combo: 13 14 34 41 76 78
Superstar: 15
### sample ###
Estrazione del: 30/05/2023 12:20 
numbers combo: 7 21 50 55 59 61
Superstar: 40


In [4]:
eurojack = Lotteria(max_num=50, max_sup=12, length=5, superstar=2)


In [5]:
for b in ['choice', 'int', 'sample']:
    print('###', b, '###')
    eurojack(backend=b, many=False)

print('\n################# many #########################\n')

for b in ['choice', 'int', 'sample']:
    print('###', b, '###')
    eurojack(backend=b, many=True)

### choice ###
Estrazione del: 30/05/2023 12:20 
numbers combo: 2 3 12 31 50
Superstar: 2 12
### int ###
Estrazione del: 30/05/2023 12:20 
numbers combo: 13 37 38 40 48
Superstar: 9 12
### sample ###
Estrazione del: 30/05/2023 12:20 
numbers combo: 27 30 32 34 35
Superstar: 7 12

################# many #########################

### choice ###
Estrazione del: 30/05/2023 12:20 
numbers combo: 6 14 15 33 38
Superstar: 1 8
### int ###
Estrazione del: 30/05/2023 12:20 
numbers combo: 12 13 19 37 49
Superstar: 9 12
### sample ###
Estrazione del: 30/05/2023 12:20 
numbers combo: 10 17 22 23 36
Superstar: 6 7


In [6]:
superenalotto(backend='choice')

Estrazione del: 30/05/2023 12:20 
numbers combo: 10 24 28 34 37 80
Superstar: 85


In [7]:
eurojack()

Estrazione del: 30/05/2023 12:20 
numbers combo: 6 26 35 41 45
Superstar: 9 12
